# Example 1: Computation of Energies and Gradients

In this example it is shown, how to run the basic energy and gradient computations. Also, the results are compared to
CASSCF implementation provided by Psi4 package.

Firstly, we specify geometry of the system given in Cartesian coordinates.

In [1]:
symbols = ['N', 'C', 'H', 'H', 'H']
coords = [
    [0.000000000000, 0.000000000000, 0.000000000000],
    [0.000000000000, 0.000000000000, 1.498047000000],
    [0.000000000000, -0.938765985000, 2.004775984000],
    [0.000000000000, 0.938765985000, 2.004775984000],
    [-0.744681452, -0.131307432, -0.634501434]
]

Subsequently, we specify the system properties.

In [2]:
n_orbs_active = 2
n_elec_active = 2
charge = 0
multiplicity = 1
basis = 'sto-3g'

Now we construct a `ProblemSet` instance containing the information about our electronic structure problem.

In [3]:
import saoovqe
import psi4

In [4]:
saoovqe.__version__

'1.1.0'

In [5]:

problem = saoovqe.problem.ProblemSet(
    symbols=symbols,
    coords=coords,
    charge=charge,
    multiplicity=multiplicity,
    n_electrons_active=n_elec_active,
    n_orbitals_active=n_orbs_active,
    basis_name=basis
)

2024-09-11 18:34:27,120 - SAOOVQE.logger - INFO - SecondQuantizedProblem was created.


The next one is a set of circuits representing orthogonal states used to construct the whole circuits
representing state vectors later.

In [6]:
import numpy as np
test1 = problem.qubit_active_hamiltonian.to_matrix()

min(np.linalg.eig(test1)[0])

(-92.67656931863549+0j)

In [7]:
initial_circuits = saoovqe.OrthogonalCircuitSet.from_problem_set(n_states=2, problem=problem)

2024-09-11 18:34:27,135 - SAOOVQE.logger - INFO - Circuits representing an orthogonal basis were created.


Another ingredient for statevector circuits constructed later will be an ansatz.

In [8]:
ansatz = saoovqe.Ansatz.from_problem_set(ansatz=saoovqe.AnsatzType.GUCCSD,
                                         problem=problem,
                                         repetitions=1,
                                         qubit_mapper=problem.fermionic_mapper)

2024-09-11 18:34:27,286 - SAOOVQE.logger - INFO - Ansatz was created.


Now we can create an instance of our SA-OO-VQE solver. Orbital-optimization doesn't have to be used,
but we do use it here, to compare with CASSCF.

In [9]:
from qiskit.primitives import Estimator

estimator = Estimator()

solver = saoovqe.SAOOVQE(estimator=estimator,
                         initial_circuits=initial_circuits,
                         ansatz=ansatz,
                         problem=problem,
                         orbital_optimization_settings={})

2024-09-11 18:34:27,290 - SAOOVQE.logger - INFO - SAOOVQE was created.


Let's create a numerical optimizer, pass it to our solver and compute our energies now.

In [10]:
from qiskit_algorithms.optimizers import SciPyOptimizer

optimizer = SciPyOptimizer('SLSQP', options={'maxiter': 500, 'ftol': 1e-8})
saoovqe_engs = solver.get_energy(optimizer)

print(saoovqe_engs)

2024-09-11 18:34:27,295 - SAOOVQE.logger - INFO - Computing energies...
2024-09-11 18:34:28,447 - SAOOVQE.logger - INFO - SA-optimized ansatz parameters: [ 0.27262107  0.27261257 -0.01552155]
2024-09-11 18:34:28,448 - SAOOVQE.logger - INFO - Starting Orbital-Optimization process...
2024-09-11 18:34:47,598 - SAOOVQE.logger - INFO - SA-optimized ansatz parameters: [ 0.19536534  0.19537057 -0.013571  ]
2024-09-11 18:34:47,599 - SAOOVQE.logger - INFO - Starting Orbital-Optimization process...
2024-09-11 18:34:59,129 - SAOOVQE.logger - INFO - SA-optimized ansatz parameters: [ 0.15570819  0.1557069  -0.01271008]
2024-09-11 18:34:59,129 - SAOOVQE.logger - INFO - Starting Orbital-Optimization process...
2024-09-11 18:35:08,485 - SAOOVQE.logger - INFO - SA-optimized ansatz parameters: [ 0.13519766  0.1351797  -0.01244071]
2024-09-11 18:35:08,485 - SAOOVQE.logger - INFO - Starting Orbital-Optimization process...
2024-09-11 18:35:15,769 - SAOOVQE.logger - INFO - SA-optimized ansatz parameters: [ 

[-92.6821175  -92.63750373]


Also, without further setting things up, we're ready to compute our gradients now!

In [11]:
import numpy as np
test1 = problem.qubit_active_hamiltonian.to_matrix()

np.linalg.eig(test1)[0]

array([-92.41426525+0.j, -92.4723376 +0.j, -92.20442739+0.j,
       -92.45870847+0.j, -92.45870847+0.j, -92.20442739+0.j,
       -92.32709418+0.j, -92.63750373+0.j, -92.70970409+0.j,
       -92.6821175 +0.j, -92.4723376 +0.j, -92.41426525+0.j,
       -91.55009137+0.j, -92.70970409+0.j, -92.70970409+0.j,
       -91.77355837+0.j])

In [12]:
saoovqe_grads = []
for state_idx in range(2):
    saoovqe_grads.append(list())
    for atom_idx in range(len(coords)):
        grad = solver.eval_eng_gradient(state_idx, atom_idx)
        print(grad)
        saoovqe_grads[state_idx].append(grad)

2024-09-11 18:36:03,773 - SAOOVQE.logger - INFO - Constructing gradient evaluators...


[array([0.16498942, 0.16491454, 0.00392844]), array([-0.16516283, -0.16508058, -0.00392605])]
[-0.22493942 -0.04882642 -0.20114204]
[array([0.16498942, 0.16491454, 0.00392844]), array([-0.16516283, -0.16508058, -0.00392605])]
[0.07458217 0.01791828 0.23628032]
[array([0.16498942, 0.16491454, 0.00392844]), array([-0.16516283, -0.16508058, -0.00392605])]
[-0.00321928  0.02466436 -0.02606309]
[array([0.16498942, 0.16491454, 0.00392844]), array([-0.16516283, -0.16508058, -0.00392605])]
[-0.00620155 -0.02588584 -0.02038731]
[array([0.16498942, 0.16491454, 0.00392844]), array([-0.16516283, -0.16508058, -0.00392605])]
[0.15977807 0.03212962 0.01131212]
[array([0.16498942, 0.16491454, 0.00392844]), array([-0.16516283, -0.16508058, -0.00392605])]
[-0.04096521 -0.02472019 -0.54302075]
[array([0.16498942, 0.16491454, 0.00392844]), array([-0.16516283, -0.16508058, -0.00392605])]
[0.06486211 0.00105818 0.51658791]
[array([0.16498942, 0.16491454, 0.00392844]), array([-0.16516283, -0.16508058, -0.003

Perfect! For now, let's set our Psi4 CASSCF solver up. Let's start with re-writing our geometry specification in Psi4-convenient
way and passing it to the toolkit.

In [13]:
import psi4
import numpy as np

def gen_formaldimine_geom_psi4(alpha, phi):
    variables = [1.498047, 1.066797, 0.987109, 118.359375, alpha, phi]

    # Create Z-matrix
    string_geo_dum = '''0 1
                    N
                    C 1 {0}
                    H 2 {1}  1 {3}
                    H 2 {1}  1 {3} 3 180
                    H 1 {2}  2 {4} 3 {5}
                    symmetry c1
                    '''.format(*variables)

    # Convert to Cartesian coordinates
    psi4.core.set_output_file('out.txt', False)
    molecule_dum = psi4.geometry(string_geo_dum)
    molecule_dum.translate(psi4.core.Vector3(-molecule_dum.x(0), -molecule_dum.y(0), -molecule_dum.z(0)))
    mol_geom_dum = molecule_dum.geometry().np * problem.unit_constants['Bohr_to_Angstrom']

    if not np.isclose(mol_geom_dum[1, 1], 0.):
        mol_geom_dum[:, [1, 2]] = mol_geom_dum[:, [2, 1]]
        mol_geom_dum[4, 0] = -mol_geom_dum[4, 0]

    string_geo = ''
    for i, e in enumerate(mol_geom_dum):
        string_geo += f'{molecule_dum.flabel(i)}, {e[0]}, {e[1]}, {e[2]}\n'

    string_geo += 'symmetry c1\n' \
                  'nocom\n' \
                  'noreorient\n'

    return string_geo

psi4.geometry(gen_formaldimine_geom_psi4(130, 80))

In [14]:
gen_formaldimine_geom_psi4(130, 80)

'N, 0.0, 0.0, 0.0\nC, 0.0, 0.0, 1.4980469996994747\nH, 0.0, -0.9387659854697812, 2.004775983750454\nH, 0.0, 0.9387659854697812, 2.004775983750454\nH, -0.7446814523077703, -0.13130743207502396, -0.6345014344827823\nsymmetry c1\nnocom\nnoreorient\n'

Now we'll specify all options for Psi4 CASSCF solver.

In [15]:
problem._active_space_transformer.prepare_active_space(16,13, occupation_alpha=None, occupation_beta=None)

In [16]:
n_orbs_frozen = len(problem.frozen_orbitals_indices)

def psi4_casscf():
    psi4.set_options({'basis': basis,
                      'DETCI_FREEZE_CORE': False,
                      'reference': 'RHF',
                      'scf_type': 'pk',
                      'num_roots': len(initial_circuits),
                      'frozen_docc': [0],
                      'restricted_docc': [n_orbs_frozen],
                      'active': [n_orbs_active],
                      'restricted_uocc': [solver.n_mo_optim - n_orbs_frozen - n_orbs_active],
                      'frozen_uocc': [problem.virtual_orbitals_indices[-1] - solver.n_mo_optim + 1],
                      'MAXITER': 1000,
                      'DIIS': False,
                      'D_CONVERGENCE': 1e-6,
                      'E_CONVERGENCE': 1e-6,
                      'S': 0,
                      'avg_states': [0, 1],
                      'avg_weights': [0.5, 0.5]})
    psi4.energy('scf', return_wfn=True)
    psi4.energy('casscf', return_wfn=True)

And run CASSCF preceded by a common SCF computation!

In [17]:
psi4.variable("CURRENT DIPOLE")

array([-0.28390182, -0.15153309,  0.43273439])

In [18]:
saoovqe_engs

array([-92.6821175 , -92.63750373])

In [19]:
psi4_casscf()

psi4_eng0 = psi4.variable('CI ROOT 0 TOTAL ENERGY')
psi4_eng1 = psi4.variable('CI ROOT 1 TOTAL ENERGY')

Cool! Now we can check, if they're similar to our SA-OO-VQE-obtained energies...

In [20]:
print(psi4_eng0 - saoovqe_engs[0])
print(psi4_eng1 - saoovqe_engs[1])

8.627179127529416e-06
-8.628366245488905e-06


Congratulations! Now it seems, that our energies are really close. Let's compute some gradients
via Psi4. In this case, we'll stick to simple stuff and utilize finite-differences approach.
Specifically, we'll obtain derivatives w.r.t. the $\alpha$ angle. So, first of all, let's generate
shifted geometries.

In [21]:
delta = 1e-5

geom_plus = gen_formaldimine_geom_psi4(alpha=130+(delta/2), phi=80)
geom_minus = gen_formaldimine_geom_psi4(alpha=130-(delta/2),phi=80)

With geometries prepared, we can evaluate the corresponding energies.

In [22]:
psi4.geometry(geom_plus)
psi4_casscf()
psi4_eng0_plus = psi4.variable('CI ROOT 0 TOTAL ENERGY')
psi4_eng1_plus = psi4.variable('CI ROOT 1 TOTAL ENERGY')

psi4.geometry(geom_minus)
psi4_casscf()
psi4_eng0_minus = psi4.variable('CI ROOT 0 TOTAL ENERGY')
psi4_eng1_minus = psi4.variable('CI ROOT 1 TOTAL ENERGY')

And finally, let's evaluate Psi4 gradients w.r.t. $\alpha$.

In [23]:
psi4_grad_0 = (psi4_eng0_plus - psi4_eng0_minus)/delta
psi4_grad_1 = (psi4_eng1_plus - psi4_eng1_minus)/delta

The gradients obtained directly from SA-OO-VQE are evaluated w.r.t. Cartesian coordinates, so we'll transform
these to internal coordinate system to be able to compare the results to each other.

In [24]:
def cartesian_to_inner(vector, R, phi, alpha):
    phi_rad = np.deg2rad(phi)
    alpha_rad = np.deg2rad(alpha)
    v_R = -(vector[0] * np.sin(phi_rad) * np.sin(alpha_rad)) - (vector[1] * np.cos(phi_rad) * np.sin(alpha_rad)) + (vector[2] * np.cos(alpha_rad))
    v_phi = -(vector[0] * R * np.cos(phi_rad) * np.sin(alpha_rad)) + (vector[1] * R * np.sin(phi_rad) * np.sin(alpha_rad))
    v_alpha = -(vector[0] * R * np.sin(phi_rad) * np.cos(alpha_rad)) - (vector[1] * R * np.cos(phi_rad) * np.cos(alpha_rad)) \
              - (vector[2] * R * np.sin(alpha_rad))

    return np.pi * np.array([v_R , v_phi, v_alpha]) / 180

bond_length = np.sqrt(sum(np.array(coords[4])**2))
saoovqe_grad_0_inner = cartesian_to_inner(saoovqe_grads[0][4], bond_length, 80, 130)
saoovqe_grad_1_inner = cartesian_to_inner(saoovqe_grads[1][4], bond_length, 80, 130)

print(saoovqe_grad_0_inner[2])
print(saoovqe_grad_1_inner[2])
print(psi4_grad_0)
print(psi4_grad_1)

0.001655015394339002
-0.0005821997288253923
0.0016561799043302015
-0.0005835232741446816


And bingo! Even the gradients seem pretty close, so they're hopefully correct :D